# Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少

In [ ]:
# 数据集加载和处理
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")


In [ ]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
    


In [ ]:
# 数据预处理
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

show_random_elements(tokenized_datasets["train"], num_examples=1)

# 使用全量数据计算，仅进行乱序

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [ ]:
# 加载模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


In [ ]:
# 模型训练器
import os
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import wandb
os.environ["WANDB_PROJECT"]="fine-tune-quickstart"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"


model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  per_device_train_batch_size=96,
                                  per_device_eval_batch_size=96,
                                  bf16=True,
                                  save_total_limit=2,
                                  hub_strategy="checkpoint",
                                  save_strategy="steps",
                                  save_steps=500,
                                  ignore_data_skip=True,
                                  report_to="wandb",
                                  logging_steps=5,
                                 )

# 定义模型评估函数
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train(resume_from_checkpoint=True)
wandb.finish()

In [ ]:
# 验证
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(500))
trainer.evaluate(small_test_dataset)


In [ ]:
trainer.save_model(f"{model_dir}/finetuned-trainer")
trainer.save_state()

In [ ]:
print(len(tokenized_datasets["test"]))